In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h5py
from sedpy.observate import load_filters, getSED
import astropy.stats
from astroquery.sdss import SDSS
from astropy import coordinates as coords
from astropy import units as u
import astropy.io.fits as fits
from astropy.modeling import models, fitting
from scipy.signal import medfilt
import scipy
from lmfit.models import GaussianModel, SkewedGaussianModel
import matplotlib.lines as mlines

import os, shutil

#from palettable.tableau import Tableau_10
#col = Tableau_10.hex_colors

#from binotools import calbino
import binotools as bt
import CalBino as cb


plt.style.use('seaborn-talk')
from matplotlib import rc_file
#rc_file(os.environ['WORK_DIR']+'/code/matplotlibrc')

plt.rc('figure', dpi=150, figsize=(12,5))

# Binospec Telluric Correction and Flux Calibration

Flux calibration code from Ben Johnson, telluric correction by `molecfit`

Run `molecfit` in `os.environ['WORK_DIR']+'/Observing/Binospec/telluric_corrections'`

Save calib files in `os.environ['BINO_DIR']+'2649_BLAS_1/calib/'`

In [2]:
#calib_dir = os.environ['BINO_DIR']+'2649_BLAS_1/calib/'

# Work on sky-subtracted data?
sky_sub = False
if sky_sub:
    skysub_lab = '_skysub'
    flux_scale = 1.
else:
    skysub_lab = ''
    flux_scale = 1.e-19

In [3]:
# Find stars on the mask

# Find the stars


def find_data():
    """
    Finds raw data
    """
    #datadir    = "/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/code/CheckRMS/Data"
    #files=sorted(bt.insensitive_glob(datadir+'/obj_clean*abs_slits_lin_m1.fits'))
    #errs=sorted(bt.insensitive_glob(datadir+'/obj_clean*abs_err_slits_lin_m1.fits'))
    datadir    = "/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/Raw/mask1"
    files=sorted(bt.insensitive_glob(datadir+'/*/obj_clean*abs_slits_lin.fits'))
    errs=sorted(bt.insensitive_glob(datadir+'/*/obj_clean*abs_err_slits_lin.fits'))
    return files,errs


spec2D_all,err2D_all = find_data()
for i,s in enumerate(spec2D_all):
    print(i,s)

stars={}
stars["index"]=[]
stars["id"]=[]
stars["ra"]=[]
stars["dec"]=[]
for ss,f in enumerate( [name for name in fits.open(spec2D_all[0]) if "ImageHDU" in str(name)]): #Get stars index
    #print(name)
    if "f_" in f.header["SLITOBJ"]:
        stars["index"].append(ss)
        stars["id"].append(f.header["SLITOBJ"])
        stars["ra"].append(f.header["ra"])
        stars["dec"].append(f.header["dec"])

#for ss,f in enumerate( [name for name in fits.open(spec2D_all[162]) if "ImageHDU" in str(name)]): #Get stars index
#    #print(name)
#    if "f_" in f.header["SLITOBJ"]:
#        stars["index"].append(ss)
#        stars["id"].append(f.header["SLITOBJ"])
#        stars["ra"].append(f.header["ra"])
#        stars["dec"].append(f.header["dec"])

print(stars)

0 /Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/Raw/Mask1/0208/obj_clean000_abs_slits_lin.fits
1 /Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/Raw/Mask1/0208/obj_clean001_abs_slits_lin.fits
2 /Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/Raw/Mask1/0208/obj_clean002_abs_slits_lin.fits
3 /Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/Raw/Mask1/0208/obj_clean003_abs_slits_lin.fits
4 /Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/Raw/Mask1/0208/obj_clean004_abs_slits_lin.fits
5 /Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/Raw/Mask1/0208/obj_clean005_abs_slits_lin.fits
6 /Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/Raw/Mask1/0208/obj_clean006_abs_slits_lin.fits
7 /Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/lar

# Telluric Functions


In [20]:
from astropy.stats import sigma_clip

def giveMaskTelluric(wave_nm):

    telluric_regions = {'O2 A band':[757., 775.],
                        'O2 B-band':[685., 694.]}

    mask_telluric = np.array([], dtype=int)
    for region in telluric_regions:
        mask_telluric = np.hstack((mask_telluric, (np.where((wave_nm > telluric_regions[region][0]) & (wave_nm < telluric_regions[region][1]))[0])))

    return mask_telluric

def std_telluric_region(args, opt_args):
    """RMS of corrected flux in region of O2 A-band"""
    
    alpha, beta = args    
    wave, flux_in, trans_std = opt_args

    mask_telluric=giveMaskTelluric(wave/10.0)
    
    flux_corr = telluric_correction(wave, flux_in, trans_std, alpha, beta)[0]

    #print(alpha,beta,np.sum(np.abs(flux_corr[mask_telluric])))
    #plt.plot(wave[mask_telluric],flux_corr[mask_telluric],".")
    #plt.title(str(alpha)+"  "+ str(beta)+"  "+str(np.nanstd(flux_corr[mask_telluric])))
    #plt.show()
    return np.nanstd(flux_corr[mask_telluric])

def scaled_transmission(wave, trans_std, alpha=0., beta=1.):
    """
    Translate and scale transmission
    """
    transmission_function = np.interp(wave-alpha, wave, trans_std)**beta   
    return transmission_function

def telluric_correction(wave, flux_in, trans_std, alpha=0., beta=1.):
    """
    Telluric correction (assuming std at same airmass as target)
    
    F_corr = F_in / T(lambda - alpha)**beta
    """
    
    # Translate and scale transmission
    transmission_function = scaled_transmission(wave, trans_std, alpha, beta) 
        
    flux_out = flux_in/transmission_function
    flux_out[flux_out == 0.] = np.nan
    
    return flux_out,transmission_function

def optimized_tellcorr(wave, flux_in, trans_std, verbose=False):
    initial_guess = [0., 1.]  # initial guess can be anything
    bnds=((-10,10),(0.,2.))
    result = scipy.optimize.minimize(std_telluric_region, initial_guess, bounds=bnds,
                                     args=[wave, flux_in*1e16, trans_std])
    if verbose:
        print(result.x)
        
    alpha = result.x[0]
    beta  = result.x[1]
    

    return telluric_correction(wave, flux_in, trans_std, alpha=alpha, beta=beta)[1], alpha, beta


####
# Gonzalo Code
####

def obtainTransmission(wave,fluxSpec,fluxTemplate):
    transm=[ fluxTemplate[i]/fluxSpec[i] for i in range(0,len(wave))]
    #transm=transm/np.median(transm)
    return transm

def TelluricMask(tellCalib,wave):
    wave=np.array(wave)
    tellCalib=np.array(tellCalib)
    wave_nm=wave/10
    telluric_regions = {'O2 A band':[757., 775.],
                    'O2 B-band':[685., 694.]}
    mask_telluric=[]
    for ss,region in enumerate(telluric_regions):
        mask_telluric.append((wave_nm < telluric_regions[region][0]) | (wave_nm > telluric_regions[region][1]))
    mask    =   np.logical_and(*mask_telluric)
    #tellCalibs= np.interp(wave,wave[mask],tellCalib[mask])

    return wave[mask],tellCalib[mask]

def runCalbino(ra,dec,data_wave,data_flux,data_err):
    calbinoStar = cb.FluxCalBino(filter_set=["sdss_g0", "sdss_r0", "sdss_i0"])
    star_mags=calbinoStar.get_star_mags(ra,dec,verbose=False)

    libwave, libflux, libparams = calbinoStar.get_library()
    best_model = calbinoStar.choose_model(star_mags, nmod=1)

    best_sed = getSED(calbinoStar.libwave, best_model, calbinoStar.filters)
    dm       = np.mean(star_mags - best_sed, axis=-1)
    conv     = np.atleast_1d(10**(-0.4 * dm))
    fluxed_model = best_model * conv[:, None]
    
    z = 0.0 # redshift of the star, if known.
    a = (1 + z)
    fluxed_model_interp = np.interp(data_wave, libwave * a, fluxed_model[0])



    #get both into the scale of the sdss magnitudes
    i0=star_mags[2] #only filter in our range 
    filteri0=load_filters(["sdss_i0"])


    waveModel,maskedModel   =   TelluricMask(fluxed_model[0],calbinoStar.libwave)
    waveData,maskedData   =   TelluricMask(data_flux,data_wave)  
    print("wave Data, masked Data:",waveData,maskedData)

    maskedData=np.nan_to_num(maskedData)
    maskedData=zero_to_median(maskedData)

    magModel= getSED(waveModel,maskedModel,filteri0)
    magData=  getSED(waveData,maskedData,filteri0)
    print("mag Data: ",magData)


    calibModel=mag2flux_dif(i0,magModel)
    calibData=mag2flux_dif(i0,magData)
    print("calib Data: ",calibData)

    fluxed_model_interp= fluxed_model_interp*calibModel
    data_flux*=calibData
    data_err*=calibData



    ##Fit curve to continuum of data and continuum of model, then subtract

    fluxModel_noContinuum=(fluxed_model_interp/calbinoStar.smooth_calibration(data_wave,fluxed_model_interp,degree=2))
    fluxData_noContinuum=(data_flux/calbinoStar.smooth_calibration(data_wave,zero_to_nan(data_flux),degree=2))

    """

    plt.plot(data_wave,fluxed_model_interp)
    #plt.plot(data_wave,medfilt(fluxed_model_interp, 251),alpha=0.8,lw=0.8,ls="--")
    plt.plot(data_wave,calbinoStar.smooth_calibration(data_wave,fluxed_model_interp,degree=2)) 
    plt.show()

    #plt.plot(data_wave,fluxed_model_interp/medfilt(fluxed_model_interp, 251))
    plt.plot(data_wave,fluxed_model_interp/calbinoStar.smooth_calibration(data_wave,fluxed_model_interp,degree=2),alpha=0.6) 
    plt.show()

    plt.plot(data_wave,zero_to_median(data_flux))
    #plt.plot(data_wave,medfilt(data_flux, 501),alpha=0.8,lw=0.8,ls="--") 
    plt.plot(data_wave,calbinoStar.smooth_calibration(data_wave,zero_to_nan(data_flux),degree=2))
    plt.show()

    plt.plot(data_wave,zero_to_median(data_flux)/medfilt(data_flux, 501))
    plt.show()


    #fluxModel_noContinuum=(fluxed_model_interp/medfilt(fluxed_model_interp, 251))
    #fluxData_noContinuum=(data_flux/medfilt(data_flux, 501))

    plt.plot(data_wave,fluxData_noContinuum/fluxModel_noContinuum)
    plt.axhline(y=1,ls="--",alpha=0.9,color="black")
    plt.ylim(-0.5,1.5)
    plt.show()

    """
    #calibration =  data_flux   /fluxed_model_interp
    calibration =   fluxData_noContinuum/fluxModel_noContinuum   




    calibration=[ 1.0 if c>1.0 else c for c in calibration]
  

    smoothed_calibration = medfilt(calibration, 101)
    
    calibration=zero_to_median(calibration)
    calibration,smoothed_calibration=zero_to_one(calibration),zero_to_one(smoothed_calibration)
    
    #plt.plot(calibration)
    #plt.plot(smoothed_calibration)
    #plt.show()


    return fluxed_model_interp,calibration,smoothed_calibration



# Fitting Functions

In [4]:
# 1D slit spectra

from scipy.optimize import minimize
from scipy import interpolate
import os

def gaussian_ML(x,flux, mu, sig,C) :
    """Gaussian"""
    return flux* np.exp( -0.5 * (x-mu)**2 / sig**2) + C

def log_likelihood_ML(theta, x, y, yerr):
    flux, mu,sig,C = theta
    model = gaussian_ML(x,flux, mu,sig,C)
    sigma2 = yerr ** 2  #sigma2 is sigma square from the likelihood stuff, not the gaussian
    return -0.5 * np.sum((y - model) ** 2 / sigma2 + np.log(sigma2))

def obtainSeeing(data,reps,rango,interval):
    seeings=[]
    for i in range(0,reps-5):
        sectionData = data[:,interval*i:interval*(i+1)] #Cut section of the data 
        collapsedData = np.nansum(sectionData,axis=1)
        collapsedData=collapsedData/np.max(collapsedData)
        #print(collapsedData)
        if np.isnan(max(collapsedData))==True:
            continue
        peak=list(collapsedData).index(max(collapsedData))

        waveX=range(0,len(collapsedData))
        nll = lambda *args: -log_likelihood_ML(*args)   #ML fit
        initial = np.array([1,peak,2,0 ])
        soln = minimize(nll, initial, args=(waveX, collapsedData, np.sqrt(np.abs(collapsedData)+0.01)))
        flux,mu,sig,C = soln.x
        
        fwhm=2.355*sig
        seeings.append(fwhm)
    #if fwhm*0.24 <0.8 or fwhm*0.24 >1.1:
        #print(rango,peak)
        #plt.plot(np.linspace(0,len(collapsedData),500),gaussian_ML(np.linspace(0,len(collapsedData),500),*soln.x))
        #plt.plot(waveX,collapsedData)
        #plt.axvline(x=mu+fwhm)
        #plt.axvline(x=mu-fwhm)
        #plt.title("Seeing: "+ str(fwhm*0.24))
        #plt.show()
    return np.nanmedian(seeings)


# Conversion Functions

In [22]:
def interpolateStar(wave,flux):
    f = interpolate.interp1d(wave, flux)
    return f

def ABtoFlux(AB):
    flux=10**-((AB+48.6)/2.5)
    return flux

def mag2flux_dif(observation,data):
    fluxCalib=ABtoFlux(observation)/ABtoFlux(data)
    return fluxCalib

def zero_to_median(lista):
    smoothed=medfilt(lista, 101)
    for i in range(0,len(lista)):
        if lista[i]==0:
            lista[i]=smoothed[i]
    return lista

def zero_to_nan(lista):
    for i in range(0,len(lista)):
        if lista[i]==0:
            lista[i]=np.nan
    return lista

def zero_to_one(lista):
    for i in range(0,len(lista)):
        if lista[i]==0:
            lista[i]=1
    return lista

# Flux Calibration Functions

In [23]:
def FluxCalib(ra,dec,data_wave,data_flux):
    #Flux must be already telluric calibrated
    calbinoStar = cb.FluxCalBino(filter_set=["sdss_g0", "sdss_r0", "sdss_i0"])
    star_mags=calbinoStar.get_star_mags(ra,dec,verbose=False)
    best_model = calbinoStar.choose_model(star_mags, nmod=1)
    best_sed = getSED(calbinoStar.libwave, best_model, calbinoStar.filters)
    
    dm       = np.mean(star_mags - best_sed, axis=-1)
    conv     = np.atleast_1d(10**(-0.4 * dm))
    fluxed_model = best_model * conv[:, None]
    fluxed_model_interp = np.interp(data_wave, calbinoStar.libwave, fluxed_model[0])

    calibration = fluxed_model_interp / data_flux


    p_init = models.Polynomial1D(degree=2)
    calibration_masked = np.ma.masked_array(calibration, mask=np.isnan(calibration))
    fit_p = fitting.LinearLSQFitter()
    p = fit_p(p_init, data_wave, calibration_masked)


    return  p(data_wave)

## Main


In [24]:
RUN=True

if RUN==True:
    PSF,PSF_ALL=[],[]
    FCAL,FCAL_ALL=[],[]
    for ii,f in enumerate(spec2D_all[:]):
        gal=fits.open(f)
        errgal=fits.open(err2D_all[ii])
        tCalibration=[]
        fCalibration=[]
        PSFs=[]
        FCALs=[]

        starIdx=stars['index']
        folder='mask1'


        for n in starIdx:
            n=n+1 # to ignore first element in each .fits
            Obj,errObj=gal[n],errgal[n]
            Yposition=int(np.round(Obj.header["SLITYPIX"]))
            rng = Yposition
            dat=Obj.data[Yposition-rng:Yposition+rng]
            err=errObj.data[Yposition-rng:Yposition+rng]
            repetitions=20
            interval=int(np.shape(dat)[1]/repetitions)
            # CALCULATE SEEING FOR EACH STAR IN EACH EXPOSURE
            seeing=obtainSeeing(dat,repetitions,rng,interval)
            PSFs.append(seeing)
            
            starSpec=Obj.data[Yposition-int(np.round(seeing)): Yposition+int(np.round(seeing))]
            starSpec=np.nansum(starSpec,axis=0)
            print("Mediana espectro estrella: ",np.nanmedian(starSpec))
            #plt.plot(starSpec)
            #plt.show()

            starSpecerr=errObj.data[Yposition-int(np.round(seeing)): Yposition+int(np.round(seeing))]
            starSpecerr=np.nansum(starSpecerr,axis=0)
            starSpec[starSpec == 0.] = np.nan
            starSpecerr[starSpecerr == 0.] = np.nan


            nstep   = len(starSpec)
            wave_nm = (Obj.header['CRVAL1'] + np.linspace(0., nstep*Obj.header['CDELT1'], nstep))
            wave=wave_nm*10

            fluxCal=FluxCalib(Obj.header['ra'] ,Obj.header['dec'],wave,starSpec)
            print("Mediana fluxcal: ",np.nanmedian(fluxCal))




            template,noisyCalib,tellCalib=   runCalbino(Obj.header['ra'] ,Obj.header['dec'],wave,starSpec*fluxCal,starSpecerr*fluxCal)
            print("telluric Calibration",tellCalib)
            tellCalib=np.nan_to_num(noisyCalib,nan=1.0,posinf=1.0,neginf=1.0) #####USE NON-SMOOTH CALIBRATION
            optimisedTellCalib=optimized_tellcorr(wave,starSpec,tellCalib)
            tellCalib=optimisedTellCalib[0]  
   

            #plt.plot(wave,starSpec,label='Standard Stars')
            #plt.plot(wave,starSpecerr,alpha=0.3,label='error')
            #plt.plot(wave,fluxTemplate,label="template")
            #plt.legend()
            #plt.show()
            #transmission =  obtainTransmission(wave,starSpec,fluxTemplate)

            #plt.plot(wave,transmission,lw=0.3)
            #wave,tellCalib   =   TelluricMask(tellCalib,wave)
            #wave,noisyCalib   =   TelluricMask(noisyCalib,wave)   
            #print(np.median(tellCalib))

            ##plt.plot(wave,noisyCalib,lw=0.8)
            #plt.plot(wave,tellCalib,lw=0.8)
            #plt.axhline(y=1,alpha=0.1,ls="--",color="black")
            #plt.xlabel("Wavelength")
            #plt.ylabel("Telluric Transmission")
            #plt.savefig("/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/code/data/large_files/Raw/mask2/CalibFigs/"+str(ii)+"_star"+str(n)+"_tellcor.pdf")
            #plt.show()


            #print(optimisedTellCalib[1:])
            
            
            #plt.plot(wave,optimisedTellCalib[0])
            #plt.show()

 




            tCalibration.append(tellCalib)
            fCalibration.append(fluxCal)

            FCALs.append(fluxCal[0])
            
            #plt.plot(wave,starSpec/tellCalib*fluxCal,label='Standard Stars')
            #plt.plot(wave,starSpecerr/tellCalib*fluxCal,alpha=0.3,label='error')
            #plt.plot(wave,template*fluxCal,label="template",alpha=0.3)
            #plt.xlabel("Wavelength")
            #plt.ylabel("Corrected Flux")
            #plt.legend()
            #plt.show()
            #plt.savefig("/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/code/data/large_files/Raw/mask2/CalibFigs/"+str(ii)+"_star"+str(n)+"_fluxcal.pdf")
            #plt.close()
        PSF.append(np.median(PSFs))
        FCAL.append(np.median(FCALs))

        PSF_ALL.append(PSFs)
        FCAL_ALL.append(FCALs)
        #print(FCAL_ALL)

        

        wave=wave
        tCalibration=np.nanmedian(tCalibration,axis=0)
        fCalibration=np.nanmedian(fCalibration,axis=0)

        for ss,ext in enumerate(gal[1:]):
            
            ext.data=ext.data/(tCalibration)*fCalibration

        print(f.split(".fits")[0]+"_fluxcal_telcor.fits")
        ordir=f.split(".fits")[0]+"_fluxcal_telcor.fits"
    



        newdir="/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/code/data/large_files/Raw/mask2/Expositions/"+folder+"/"+ordir.split("/")[-2]
        isExist = os.path.exists(newdir)
        if not isExist:
            # Create a new directory because it does not exist 
            os.makedirs(newdir)
            print("The new directory is created!")



        print(newdir)
        gal.writeto(newdir+"/"+ordir.split("/")[-1],overwrite=True)

        for ss,ext in enumerate(errgal[1:]):
            ext.data=ext.data/(tCalibration)*fCalibration

        
        print(err2D_all[ii].split(".fits")[0]+"_fluxcal_telcor.fits")
        ordir=err2D_all[ii].split(".fits")[0]+"_fluxcal_telcor.fits"
        newdir="/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/code/data/large_files/Raw/mask2/Expositions/"+folder+"/"+ordir.split("/")[-2]
        print(newdir)
        errgal.writeto(newdir+"/"+ordir.split("/")[-1],overwrite=True)










        

#nstep   = spec1D.shape[1]
#wave_nm = header['CRVAL1'] + np.linspace(0., nstep*header['CDELT1'], nstep)
            
        
 





Mediana espectro estrella:  123.98253
Mediana fluxcal:  3.3122687489013547e-18
wave Data, masked Data: [ 6760.          6760.62011011  6761.24022022 ... 10250.59980663
 10251.21991674 10251.84002685] [nan nan nan ... nan nan nan]
mag Data:  [16.3539302]
calib Data:  [0.97442424]
telluric Calibration [1. 1. 1. ... 1. 1. 1.]
Mediana espectro estrella:  48.605473
Mediana fluxcal:  3.512766836001437e-18
wave Data, masked Data: [ 6760.          6760.62011011  6761.24022022 ... 10250.59980663
 10251.21991674 10251.84002685] [nan nan nan ... nan nan nan]
mag Data:  [17.3264828]
calib Data:  [0.97651995]
telluric Calibration [0.91163343 0.91260806 0.93475956 ... 1.         1.         1.        ]
/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/code/data/large_files/Raw/mask1/Expositions/mask1/0208/obj_clean000_abs_slits_lin_fluxcal_telcor.fits
/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/code/data/large_files/Raw/mask2/Expositions/mask1/0208
/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/code/data/l

<ipython-input-20-f1d64efdf2aa>:34: RuntimeWarning: invalid value encountered in power
  transmission_function = np.interp(wave-alpha, wave, trans_std)**beta


Mediana fluxcal:  4.317855310759141e-18
wave Data, masked Data: [ 6760.          6760.62011011  6761.24022022 ... 10250.59980663
 10251.21991674 10251.84002685] [nan nan nan ... nan nan nan]
mag Data:  [17.31623137]
calib Data:  [0.96734312]
telluric Calibration [0.75430117 0.98473165 0.98921068 ... 1.         1.         1.        ]
/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/code/data/large_files/Raw/mask1/Expositions/mask1/0628/obj_clean000_abs_slits_lin_fluxcal_telcor.fits
/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/code/data/large_files/Raw/mask2/Expositions/mask1/0628
/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/code/data/large_files/Raw/mask1/Expositions/mask1/0628/obj_clean000_abs_err_slits_lin_fluxcal_telcor.fits
/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/code/data/large_files/Raw/mask2/Expositions/mask1/0628
Mediana espectro estrella:  90.87087
Mediana fluxcal:  5.096264964760326e-18
wave Data, masked Data: [ 6760.          6760.62011011  6761.24022022 ... 10250.59980663
 

<ipython-input-20-f1d64efdf2aa>:34: RuntimeWarning: invalid value encountered in power
  transmission_function = np.interp(wave-alpha, wave, trans_std)**beta


Mediana espectro estrella:  39.48099
Mediana fluxcal:  4.550126370199454e-18
wave Data, masked Data: [ 6760.          6760.62011011  6761.24022022 ... 10250.59980663
 10251.21991674 10251.84002685] [nan nan nan ... nan nan nan]
mag Data:  [17.32922131]
calib Data:  [0.9789861]
telluric Calibration [0.38113454 0.99382445 0.99411498 ... 1.         1.         1.        ]
/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/code/data/large_files/Raw/mask1/Expositions/mask1/0628/obj_clean001_abs_slits_lin_fluxcal_telcor.fits
/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/code/data/large_files/Raw/mask2/Expositions/mask1/0628
/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/code/data/large_files/Raw/mask1/Expositions/mask1/0628/obj_clean001_abs_err_slits_lin_fluxcal_telcor.fits
/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/code/data/large_files/Raw/mask2/Expositions/mask1/0628
Mediana espectro estrella:  138.88104
Mediana fluxcal:  3.3523047632680587e-18
wave Data, masked Data: [ 6760.          6760.620110

# SEEING and FWHM figures

In [21]:
starIdx=stars['index']

PSF,PSF_ALL=[],[]
FCAL,FCAL_ALL=[],[]
for ii,f in enumerate(spec2D_all[:]):
    gal=fits.open(f)
    #errgal=fits.open(err2D_all[ii])
    PSFs=[]
    folder=""
    if len(gal)==171:
        starIdx=stars['index'][:2]
        folder='mask2_1'
    if len(gal)==152:
        starIdx=stars['index'][2:4]
        folder="mask2_2"
    if len(gal)==154:
        starIdx=stars['index'][4:6]
        folder="mask2_3"
    if len(gal)==174:
        starIdx=stars['index'][6:8]
        folder="mask1"
        print


    for n in starIdx:
        n=n+1 # to ignore first element in each .fits
        #Obj,errObj=gal[n],errgal[n]
        Obj=gal[n]

        Yposition=int(np.round(Obj.header["SLITYPIX"]))
        rng = 15 
        if rng>=Yposition:
            rng=Yposition
        dat=Obj.data[Yposition-rng:Yposition+rng]
        #err=errObj.data[Yposition-rng:Yposition+rng]
        repetitions=20
        interval=int(np.shape(dat)[1]/repetitions)
        print(f)
        seeing=obtainSeeing(dat,repetitions,rng,interval)
        PSFs.append(seeing)
        PSF.append(np.median(PSFs))
        PSF_ALL.append(PSFs)

/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/Raw/Mask1/0208/obj_clean000_abs_slits_lin.fits
/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/Raw/Mask1/0208/obj_clean000_abs_slits_lin.fits
/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/Raw/Mask1/0208/obj_clean001_abs_slits_lin.fits
/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/Raw/Mask1/0208/obj_clean001_abs_slits_lin.fits
/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/Raw/Mask1/0208/obj_clean002_abs_slits_lin.fits
/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/Raw/Mask1/0208/obj_clean002_abs_slits_lin.fits
/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/Raw/Mask1/0208/obj_clean003_abs_slits_lin.fits
/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/Raw/Mas

In [24]:
np.array(PSF_ALL)*0.24

array([[0.97706207, 0.98087369],
       [0.97706207, 0.98087369],
       [0.93600596, 0.9314803 ],
       [0.93600596, 0.9314803 ],
       [0.89516502, 0.88871968],
       [0.89516502, 0.88871968],
       [0.84535566, 0.8479299 ],
       [0.84535566, 0.8479299 ],
       [0.79757376, 0.78881886],
       [0.79757376, 0.78881886],
       [0.86564267, 0.86185896],
       [0.86564267, 0.86185896],
       [0.75733345, 0.75243457],
       [0.75733345, 0.75243457],
       [0.78286507, 0.78117067],
       [0.78286507, 0.78117067],
       [0.97110629, 0.95867898],
       [0.97110629, 0.95867898],
       [1.0185437 , 1.00962608],
       [1.0185437 , 1.00962608],
       [1.06335071, 1.06678611],
       [1.06335071, 1.06678611],
       [1.01794391, 1.00038141],
       [1.01794391, 1.00038141],
       [1.05737024, 1.05578721],
       [1.05737024, 1.05578721],
       [0.74954806, 0.73873128],
       [0.74954806, 0.73873128],
       [0.88870239, 0.884953  ],
       [0.88870239, 0.884953  ],
       [0.

# WHICH EXPOSITIONS WE DONT TAKE?

In [29]:
Seeing_lim=1.2
Exceptions=[]
for f,seeing in zip(spec2D_all,PSF_ALL):
    print(f,np.mean(seeing)*0.24)
    if np.mean(np.array(seeing)*0.24) > Seeing_lim:
        Exceptions.append(f)
#np.save("/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/code/files/BadSeeing_Exposures.npy",Exceptions)

    

/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/Raw/Mask1/0208/obj_clean000_abs_slits_lin.fits 0.9789678764780427
/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/Raw/Mask1/0208/obj_clean001_abs_slits_lin.fits 0.9789678764780427
/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/Raw/Mask1/0208/obj_clean002_abs_slits_lin.fits 0.9337431292543942
/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/Raw/Mask1/0208/obj_clean003_abs_slits_lin.fits 0.9337431292543942
/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/Raw/Mask1/0208/obj_clean004_abs_slits_lin.fits 0.8919423511490463
/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/Raw/Mask1/0208/obj_clean005_abs_slits_lin.fits 0.8919423511490463
/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/Raw/Mask1/0208/obj_clea

In [14]:
Exceptions

['/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/Raw/Mask1/0704/obj_clean001_abs_slits_lin.fits',
 '/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/Raw/Mask1/0704/obj_clean002_abs_slits_lin.fits',
 '/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/Raw/Mask1/0704/obj_clean003_abs_slits_lin.fits',
 '/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/Raw/Mask1/0704/obj_clean004_abs_slits_lin.fits']

In [ ]:
def flatten(t):
    return np.array([item for sublist in t for item in sublist])


pixscale=0.24

plt.hist(np.array(PSF)*pixscale,bins=12)
plt.ylabel("N Expositions")
plt.xlabel("FWHM in arcsec")
plt.title("Median of the two stars, ALL expositions")
plt.show()

cut1,cut2,cut3=66,122,162

plt.hist(np.transpose(PSF_ALL[0:cut1])[0]*pixscale,bins=10,alpha=0.5)
plt.hist(np.transpose(PSF_ALL[0:cut1])[1]*pixscale,bins=10,alpha=0.5)
plt.hist(np.array(PSF[0:cut1])*pixscale,bins=12,color="gray",ls="--",edgecolor="black",lw=2.0,alpha=0.6)
plt.ylabel("N Expositions")
plt.xlabel("FWHM in arcsec")
plt.title("mask 2_1")
plt.show()

plt.hist(np.transpose(PSF_ALL[cut1:cut2])[0]*pixscale,bins=10,alpha=0.5)
plt.hist(np.transpose(PSF_ALL[cut1:cut2])[1]*pixscale,bins=10,alpha=0.5)
plt.hist(np.array(PSF[cut1:cut2])*pixscale,bins=12,color="gray",ls="--",edgecolor="black",lw=2.0,alpha=0.6)
plt.ylabel("N Expositions")
plt.xlabel("FWHM in arcsec")
plt.title("mask 2_2")
plt.show()

plt.hist(np.transpose(PSF_ALL[cut2:cut3])[0]*pixscale,bins=10,alpha=0.5)
plt.hist(np.transpose(PSF_ALL[cut2:cut3])[1]*pixscale,bins=10,alpha=0.5)
plt.hist(np.array(PSF[cut2:])*pixscale,bins=12,color="gray",ls="--",edgecolor="black",lw=2.0,alpha=0.6)
plt.ylabel("N Expositions")
plt.xlabel("FWHM in arcsec")
plt.title("mask 2_3")
plt.show()

plt.hist(np.transpose(PSF_ALL[cut3:])[0]*pixscale,bins=10,alpha=0.5)
plt.hist(np.transpose(PSF_ALL[cut3:])[1]*pixscale,bins=10,alpha=0.5)
plt.hist(np.array(PSF[cut2:])*pixscale,bins=12,color="gray",ls="--",edgecolor="black",lw=2.0,alpha=0.6)
plt.ylabel("N Expositions")
plt.xlabel("FWHM in arcsec")
plt.title("mask 1")
plt.show()

"""

plt.hist(FCAL,bins=20)
plt.ylabel("N Expositions")
plt.xlabel("Flux Cal")
plt.title("Median of the two stars, ALL expositions")
plt.show()

plt.hist(np.transpose(FCAL_ALL[0:cut1])[0],bins=5)
plt.hist(np.transpose(FCAL_ALL[0:cut1])[1],bins=5)
plt.hist(np.array(FCAL[0:cut1]),bins=10,color="gray",ls="--",edgecolor="black",lw=2.0,alpha=0.6)
plt.ylabel("N Expositions")
plt.xlabel("Flux Cal")
plt.title("mask 2_1")
plt.show()

plt.hist(np.transpose(FCAL_ALL[cut1:cut2])[0],bins=5)
plt.hist(np.transpose(FCAL_ALL[cut1:cut2])[1],bins=5)
plt.hist(np.array(FCAL[cut1:cut2]),bins=10,color="gray",ls="--",edgecolor="black",lw=2.0,alpha=0.6)
plt.ylabel("N Expositions")
plt.xlabel("Flux Cal")
plt.title("mask 2_2")
plt.show()

plt.hist(np.transpose(FCAL_ALL[cut2:])[0],bins=5)
plt.hist(np.transpose(FCAL_ALL[cut2:])[1],bins=5)
plt.hist(np.array(FCAL[cut2:]),bins=10,color="gray",ls="--",edgecolor="black",lw=2.0,alpha=0.6)
plt.ylabel("N Expositions")
plt.xlabel("Flux Cal")
plt.title("mask 2_3")
plt.show()
"""


In [ ]:
#print(spec2D_all[66:67])
counter=0
first=[]
for n,f in enumerate(spec2D_all[0:1]):
    gal=fits.open(f)
    for i in gal[1:]:
        if "z" in i.header["SLITOBJ"]:
            counter+=1
            first.append(i.header["SLITOBJ"])
            #print(i.header["SLITOBJ"])
print(counter)

second=[]
counter=0
for n,f in enumerate(spec2D_all[70:71]):
    gal=fits.open(f)
    for i in gal[1:]:
        if "z" in i.header["SLITOBJ"]:
            counter+=1
            second.append(i.header["SLITOBJ"])
            #print(i.header["SLITOBJ"])
print(counter)

third=[]
counter=0
for n,f in enumerate(spec2D_all[131:132]):
    gal=fits.open(f)
    for i in gal[1:]:
        if "z" in i.header["SLITOBJ"]:
            counter+=1
            third.append(i.header["SLITOBJ"])
            #print(i.header["SLITOBJ"])
print(counter)

for i in first:
    for j in second:
        if i==j:
            print(i,j)

for i in first:
    for j in third:
        if i==j:
            print(i,j)

for i in second:
    for j in third:
        if i==j:
            print(i,j)